### **TÍTULO**

Import libraries

In [14]:
import os
from PIL import Image
import pandas as pd
import cv2

In [28]:
#def load_images(path):
    #'''Funtion to load images using PIL'''
    #images = []
    #names = []
    #for file in os.listdir(path):
        #final_path = os.path.join(path, file)
        #image = Image.open(final_path)
        #images.append(image)
        #names.append(file)

    #return images, names

def load_images_opencv(path):
    '''Funtion to load images using OpenCV'''
    images = []
    names = []
    for file in os.listdir(path):
        img_path = os.path.join(path, file)
        image = cv2.imread(img_path)
        images.append(image)
        names.append(file)
    return images, names


def load_labels(path):
    '''Function to load labels'''
    labels = {}
    for file in os.listdir(path): 
        final_path = os.path.join(path, file)
        with open(final_path, 'r') as f: 
            # Parse each line as a list of floats (assuming labels are space-separated)
            label_data = [list(map(float, line.strip().split())) for line in f.readlines()]
            labels[file] = label_data
    return labels



def resize_images_and_labels(images, labels, size=(640, 640)):
    '''Function for resizing images and adjusting labels including keypoints'''
    resized_images = []
    resized_labels = {}

    for img, name in zip(images, labels.keys()):
        h, w = img.shape[:2]
        resized_img = cv2.resize(img, size)
        resized_images.append(resized_img)

        # Adjust the label coordinates to the new image size
        new_labels = []
        for label in labels[name]:
            class_id = label[0]
            x_center = label[1] * size[0] / w  # Adjust x_center
            y_center = label[2] * size[1] / h  # Adjust y_center
            box_width = label[3] * size[0] / w  # Adjust width
            box_height = label[4] * size[1] / h  # Adjust height

            # Adjust keypoints (if present)
            keypoints = []
            if len(label) > 5:
                for i in range(5, len(label), 2):  # Iterate over keypoint pairs
                    kp_x = label[i] * size[0] / w  # Adjust keypoint x
                    kp_y = label[i+1] * size[1] / h  # Adjust keypoint y
                    keypoints.extend([kp_x, kp_y])

            # Combine adjusted values
            new_label = [class_id, x_center, y_center, box_width, box_height] + keypoints
            new_labels.append(new_label)

        resized_labels[name] = new_labels

    return resized_images, resized_labels


#Check if image are in colour scale
def convert_to_grayscale(images):
    '''Function to convert images to gray scale'''
    grayscale_images = []
    for img in images:
        grayscale_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        grayscale_images.append(grayscale_img)
    return grayscale_images


def normalize_images(images):
    '''Function to normalize pixel values (0 - 1)'''
    normalized_images = []
    for img in images:
        normalized_img = img / 255.0  
        normalized_images.append(normalized_img)
    return normalized_images


def save_images_and_labels(images, names, labels, save_path_img, save_path_lbl):
    '''Function to save processed images and adjusted labels'''
    for img, name in zip(images, names):
        # Save the image
        cv2.imwrite(os.path.join(save_path_img, name), img)

        # Save the corresponding label
        label_name = name.replace('.jpg', '.txt')  # Assuming image name is .jpg and label is .txt
        with open(os.path.join(save_path_lbl, label_name), 'w') as f:
            for label in labels[label_name]:
                f.write(' '.join(map(str, label)) + '\n')



### Import data

In [6]:
train_images_path = r"..\1-Data\BoneFractureYolo8\train\images"
train_labels_path = r"..\1-Data\BoneFractureYolo8\train\labels"
test_images_path = r"..\1-Data\BoneFractureYolo8\test\images"
test_labels_path = r"..\1-Data\BoneFractureYolo8\test\labels"
valid_images_path = r"..\1-Data\BoneFractureYolo8\valid\images"
valid_labels_path = r"..\1-Data\BoneFractureYolo8\valid\labels"

In [29]:
#Train dataset
train_images, train_names = load_images_opencv(train_images_path)
train_labels = load_labels(train_labels_path)

#Valid dataset
valid_images, valid_names = load_images_opencv(valid_images_path)
valid_labels = load_labels(valid_labels_path)

#Test
test_images, test_names = load_images_opencv(test_images_path)
test_labels =load_labels(test_labels_path)

### Processing images

In [31]:
#Resizing images
train_resized_images, train_resized_labels = resize_images_and_labels(train_images, train_labels)
test_resized_images, test_resized_labels = resize_images_and_labels(test_images, test_labels)
valid_resized_images, valid_resized_labels = resize_images_and_labels(valid_images, valid_labels)

Importar imágenes
Procesar imágenes procesadas
Importar el yaml con la simágenes porcesadas
Entrenar modeleo preentrenado

Unir imagenes y etiquetas
datos = []

for nombre in nombres_imagenes:
    nombre_txt = nombre.replace('.jpg', '.txt').replace('.png', '.txt')
    etiqueta = etiquetas.get(nombre_txt, 'Sin etiqueta')
    datos.append({
        'nombre_imagen': nombre,
        'imagen': nombre,  # Aquí podrías poner la ruta si prefieres
        'etiqueta': etiqueta
    })

df = pd.DataFrame(datos)






# Guardar imágenes
def save_images_opencv(images, names, save_path):
    for img, name in zip(images, names):
        cv2.imwrite(os.path.join(save_path, name), img)

# Cargar imágenes usando OpenCV
def load_images_opencv(path):
    images = []
    names = []
    for file in os.listdir(path):
        img_path = os.path.join(path, file)
        image = cv2.imread(img_path)
        images.append(image)
        names.append(file)
    return images, names

# Ejemplo de preprocesado completo
train_images_path = r"..\1-Data\BoneFractureYolo8\train\images"
train_images, train_names = load_images_opencv(train_images_path)

# Redimensionar, convertir a escala de grises, normalizar
train_images_resized = resize_images_opencv(train_images)
train_images_gray = convert_to_grayscale_opencv(train_images_resized)
train_images_normalized = normalize_images_opencv(train_images_gray)

# Guardar las imágenes preprocesadas
save_images_opencv(train_images_normalized, train_names, "path_to_save_preprocessed_images")